In [1]:
#Connectings to Milvus, BERT and Postgresql
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility
import pymysql

connections.connect(host='localhost', port='19530')
conn = pymysql.connect(host='localhost', user='root', port=3306, password='123456', database='mysql',local_infile=True)
cursor = conn.cursor()

In [2]:
TABLE_NAME = 'question_answering'

#Deleting previouslny stored table for clean run
if utility.has_collection(TABLE_NAME):
    collection = Collection(name=TABLE_NAME)
    collection.drop()

field1 = FieldSchema(name="id", dtype=DataType.INT64, descrition="int64", is_primary=True, auto_id=True)
field2 = FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, descrition="float vector",dim=768, is_primary=False)
schema = CollectionSchema(fields=[field1, field2], description="collection description")
collection = Collection(name=TABLE_NAME, schema=schema)

In [3]:
default_index = {"index_type": "IVF_FLAT", "metric_type": 'IP', "params": {"nlist": 200}}
collection.create_index(field_name="embedding", index_params=default_index)

Status(code=0, message=)

In [4]:
#Deleting previouslny stored table for clean run
drop_table = "DROP TABLE IF EXISTS " + TABLE_NAME + ";"
cursor.execute(drop_table)

try:
    sql = "CREATE TABLE if not exists " + TABLE_NAME + " (id TEXT, question TEXT, answer TEXT);"
    cursor.execute(sql)
    print("create MySQL table successfully!")
except Exception as e:
    print("can't create a MySQL table: ", e)

create MySQL table successfully!


In [5]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.preprocessing import normalize

model = SentenceTransformer(r'D:\test\paraphrase-multilingual-mpnet-base-v2')

# Get questions and answers.
data = pd.read_csv(r'D:\test\test\test\question_answer1.csv', encoding='latin1')
question_data = data['question'].tolist()
answer_data = data['answer'].tolist()

sentence_embeddings = model.encode(question_data)
sentence_embeddings = normalize(sentence_embeddings).tolist()

In [6]:
mr = collection.insert([sentence_embeddings])
ids = mr.primary_keys
print(len(ids))

# status, ids = milv.insert(collection_name=TABLE_NAME, records=sentence_embeddings)
# print(status)

1000


In [7]:
import os 

# Combine the id of the vector and the question data into a list
def format_data(ids, question_data, answer_data):
    data = []
    for i in range(len(ids)):
        value = (str(ids[i]), question_data[i], answer_data[i])
        data.append(value)
    return data

def load_data_to_mysql(cursor, conn, table_name, data):
    sql = "insert into " + table_name + " (id,question,answer) values (%s,%s,%s);"
    try:
        cursor.executemany(sql, data)
        conn.commit()
        print("MYSQL loads data to table: {} successfully".format(table_name))
    except Exception as e:
        print("MYSQL ERROR: {} with sql: {}".format(e, sql))
        
load_data_to_mysql(cursor, conn, TABLE_NAME, format_data(ids, question_data, answer_data))


MYSQL loads data to table: question_answering successfully


In [8]:
SEARCH_PARAM = {'nprobe': 40}

query_vec = []

question = "What is a movie? Where did movies originate from?"

query_embeddings = []
embed = model.encode(question)
embed = embed.reshape(1,-1)
embed = normalize(embed)
query_embeddings = embed.tolist()

collection.load()

search_params = {"metric_type": 'IP', "params": {"nprobe": 16}}

results = collection.search(query_embeddings, anns_field="embedding", param=search_params, limit=5)

# status, results = milv.search(collection_name=TABLE_NAME, query_records=query_embeddings, top_k=5, params=SEARCH_PARAM)


In [9]:
ids = [str(x.id) for x in results[0]]

def search_by_milvus_ids(cursor, ids, table_name):
    str_ids = str(ids).replace('[', '').replace(']', '')
    sql = "select question from " + table_name + " where id in (" + str_ids + ") order by field (id," + str_ids + ");"
    try:
        cursor.execute(sql)
        results = cursor.fetchall()
        results = [res[0] for res in results]
        return results
    except Exception as e:
        print("MYSQL ERROR: {} with sql: {}".format(e, sql))

similar_questions = search_by_milvus_ids(cursor, ids, TABLE_NAME)

distances = [x.distance for x in results[0]]

res = dict(zip(similar_questions, distances))

print('There are similar questions in the database, here are the closest matches:\n{}'.format(res))


There are similar questions in the database, here are the closest matches:
{'What is a movie? Where did movies originate from?': 0.9999998807907104, 'What is film art?': 0.6886543035507202, 'What is a movie song?': 0.6777299046516418, 'What is film language?': 0.6602007150650024, 'What is film composition?': 0.6433337330818176}


In [10]:
sql = "select answer from " + TABLE_NAME + " where question = '" + similar_questions[0] + "';"

cursor.execute(sql)
rows=cursor.fetchall()
print("Question:")
print(question)
print("Answer:")
print(rows[0][0])

Question:
What is a movie? Where did movies originate from?
Answer:
According to the principle of visual persistence, photography is used to capture images of external objects on film, creating a moving image on the screen through the projection process to express certain content. Scientific experiments have shown that after a certain image disappears, the human eye can still retain the object on the film for about 0.1-0.4 seconds. The movies we currently see are based on this "visual persistence" theory, rotating at a constant speed of 24 frames per second, causing a series of static images to fill the gaps between the images due to the effect of visual persistence, creating a continuous visual impression and producing a realistic sense of motion. By 1894-1895, the Lumiere brothers in France were able to create a film that can capture images. Like a projector projected on a white screen, real movies were finally born. Early movies were all black and white silent films, but attempts to

In [32]:
import tkinter as tk

def search_and_display():
    question = question_text.get("1.0", "end-1c")  # 获取文本框中的内容
    query_embeddings = model.encode([question])
    query_embeddings = normalize(query_embeddings).tolist()
    
    collection.load()
    search_params = {"metric_type": 'IP', "params": {"nprobe": 16}}
    results = collection.search(query_embeddings, anns_field="embedding", param=search_params, limit=5)
    
    ids = [str(x.id) for x in results[0]]
    similar_questions = search_by_milvus_ids(cursor, ids, TABLE_NAME)
    distances = [x.distance for x in results[0]]
    res = dict(zip(similar_questions, distances))
    text.insert(tk.END, '\n------------------------------------------------------------------------------------------------------------------------\n')
    text.insert(tk.END, 'There are similar questions in the database, here are the closest matches:\n{}'.format(res))

    sql = "select answer from " + TABLE_NAME + " where question = '" + similar_questions[0] + "';"
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    text.insert(tk.END, "\n\nQuestion:\n{}\n\n".format(question))
    text.insert(tk.END, "Answer:\n{}".format(rows[0][0]))
    text.insert(tk.END, '\n------------------------------------------------------------------------------------------------------------------------\n')

def clear_text():
    text.delete("1.0", tk.END)  # 清空文本框内容

root = tk.Tk()
root.title("Question Answering System")

# 创建标签和文本框用于输入问题
question_label = tk.Label(root, text="Enter your question:")
question_label.pack()
question_text = tk.Text(root, width=100, height=5)  # 设置文本框的宽度和高度
question_text.pack()

# 创建按钮容器
button_frame = tk.Frame(root)
button_frame.pack(pady=5)  # 添加垂直填充

# 创建按钮用于触发搜索
search_button = tk.Button(button_frame, text="Search", command=search_and_display)
search_button.pack(side=tk.LEFT, padx=60)  # 添加水平填充

# 创建清空按钮
clear_button = tk.Button(button_frame, text="Clear", command=clear_text)
clear_button.pack(side=tk.LEFT, padx=60)  # 添加水平填充

# 创建文本框用于显示问题和答案
text = tk.Text(root, width=120, height=50) 
text.pack()

root.mainloop()
